In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets
!pip install triton
!pip install jsonlines

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-xe0_llv7/unsloth_2f79e10edb49468abd0f3d39273d9b3c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-xe0_llv7/unsloth_2f79e10edb49468abd0f3d39273d9b3c
  Resolved https://github.com/unslothai/unsloth.git to commit a0acecb50f39d9b62a144684be9ed9e3c3755a1f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#Bibliotecas Import
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from google.colab import drive
import json
import numpy as np

#Importando bibliotecas para realizar o fine tuning
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer

# 1 - Escolha o Dataset

O AmazonTitles-1.3MM é um conjunto de dados que reúne buscas feitas por usuários e os títulos dos produtos relacionados encontrados na Amazon. A conexão entre as buscas e os produtos é baseada nas descrições e nas ações dos usuários, como cliques, visualizações, compras ou avaliações.

In [ ]:
drive.mount('/content/drive',force_remount=True)
dataset_amazon = '/content/drive/MyDrive/FIAP - Fase 3/trn.json'

Mounted at /content/drive


# 2 - Preparação do Dataset

Abrindo o arquivo json e lendo como df do pandas
*Nota: o arquivo json esta malformatado em algumas linhas e para isto, foi incluido uma logica para descartar as linhas mal formatadas.*

In [ ]:
data =[]
with open(dataset_amazon, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            obj = json.loads(line)  # Tentar carregar como JSON
            data.append(obj)
        except json.JSONDecodeError as e:
            print(f"Linha com erro: {e}")

Conversão do json em um DataFrame e realizando a análise dos valores nulos

In [ ]:
# Transformar a lista de objetos JSON válidos em um DataFrame
df = pd.DataFrame(data)

#Primeiras linhas do df
df.head()

# Substituir valores brancos por NaN para depois contar e filtrar os textos sem valores preenchidos
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# contagem de linhas do df
print(df.count())


# Contar valores nulos por coluna
valores_nulos_title = valores_nulos_coluna2 = df['title'].isnull().sum()
valores_nulos_content = valores_nulos_coluna2 = df['content'].isnull().sum()

print(valores_nulos_title) # encontrado 126834 linhas com valores nulos
print(valores_nulos_content) # encontrado 749901 linhas com valores nulos

uid           2248619
title         2121785
content       1498718
target_ind    2248619
target_rel    2248619
dtype: int64
126834
749901


Tratamento dos dados nulos

In [ ]:
#remove as colunas vazias de ambas as colunas e salva em um novo df
df_fine_tuning = df[['title', 'content']].dropna()

df_fine_tuning.count() #1390403 linhas com titulo e conteudo preenchidas

df_fine_tuning = df_fine_tuning.drop_duplicates(subset=['title', 'content'])

print(df_fine_tuning.count()) #quantidade de linhas sem duplicatas e valores nulos 1367131

del df #deleta o df antigo



title      1367131
content    1367131
dtype: int64


Transformando o dataframe para ser processado no formato que o fine-tuning espera (*instrução do prompt, texto a ser avaliado e output do resultado*).
No caso vamos gerar a descrição de um produto, baseado em um titulo que será informado pelo usuário. Por conta disto, vamos definir que o input do usuário será: "*Elaborate a description of product based on the title provided.*"


In [ ]:
instructions =[]
title =[]
content =[]

for index, row in df_fine_tuning.iterrows():
  instructions.append('Elaborate a description of product based on the title provided.')
  title.append(row['title'])
  content.append(row['content'])

data_fine_tunning ={
    'instruction': instructions,
    'title': title,
    'content': content
}

output_dataset = '/content/drive/MyDrive/FIAP - Fase 3/fine_tuning.json'

with open(output_dataset, 'w') as output_file:
    json.dump(data_fine_tunning, output_file, indent=4)

del df_fine_tuning



# 3- Chamada do Foundation Model

O Foundation Model que utilizaremos será o LLAMA 3

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", #Modelo escolhido
    max_seq_length = 2048, #Numero maximo de tokens retornados pelo Modelo
    dtype = None,
    load_in_4bit = True, # diminuir a quantidade de casas decimais dos embbedings para 4 para economizar recursos computacional
)

==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Executando o modelo sem o fine-tuning para avaliar a diferença do resultado gerado

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 1000,
    use_rslora = False,
    loftq_config = None,
)

FastLanguageModel.for_inference(model)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Title:
{}

### Content:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Elaborate a description of product based on the title provided.",
        "Golden Hatchet", # input
        "",
    )
], return_tensors = "pt").to("cuda")#pytorch tensor

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Elaborate a description of product based on the title provided.

### Title:
Golden Hatchet

### Content:
A golden hatchet that can cut through anything and everything. It's been a tool of the gods since the beginning of time. It's been used to chop down the tallest trees, carve out the deepest caves, and carve out the greatest monuments. It's been used to cut down the tallest mountains, carve out the deepest valleys, and carve out the greatest rivers. It's been used to chop down the tallest buildings, carve out the deepest lakes, and carve out the greatest cities. It's been used to chop down the tallest forests, carve out the deepest oceans, and carve out the greatest deserts. It's been used to


# 4 - Execução do Fine-Tuning

Utilizando o dataset preparado anteriormente para a execução do fine-tuning

In [ ]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["title"]
    outputs      = examples["content"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


dataset = load_dataset("json", data_files=output_dataset, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1367131 [00:00<?, ? examples/s]

Definindo os parametros para o treinamento, nos comentários algumas explicações do motivo da utilziação dos parâmetros

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2, #testes com numeros maiores, o treinamento foi mais rápido, mas as respostas foram imprecisas ou se repetiam
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2, #devido a limitação dos recursos de hardware, aumentar esse parametro ocorre um estouro de memória
        gradient_accumulation_steps = 4, #devido a limitação dos recursos de hardware, aumentar esse parametro ocorre um estouro de memória
        warmup_steps = 5, #devido a limitação dos recursos de hardware, aumentar os passos de aquecimento ocorre um estouro de memória
        max_steps = 60,
        learning_rate = 0.00002,
        fp16 = not is_bfloat16_supported(), #função para validar se a GPU/hardrware suporta ponmto flutuante de 16 bits. Caso não, retorna true e trabalha com mixed precision, melhorando o desempenho e reduzindo o custo de memoria
        bf16 = is_bfloat16_supported(),#similar ao comentário acima, porem com bfloat16
        logging_steps = 1, #optamos por 1 para visualizar com mais detalhes os valores do traing loss durante as épocas
        optim = "adamw_8bit", # esse otimizador é eficiente em termos de memória para grandes modelos. Define o otimizador para ajustar os pesos das redes neurais (Adam com decaimento de peso). o ajuste entre esta metrica e o learning rate é fundamental para ter uma boa taxa de aprendizagem.
        weight_decay = 0.06, #Um fator de regularização que evita que os pesos do modelo cresçam demai
        lr_scheduler_type = "linear",#controla como a taxa de aprendizado varia durante o treinamento
        seed = 3407,#estado aleatorio do dataset, util para sempre manter e usar a mesma foto
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1367131 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Treinamento do modelo

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,367,131 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.662100
2,3.922700
3,3.842400
4,3.696200
5,4.002400
6,3.671700
7,3.897400
8,3.729300
9,3.449300
10,3.604900


# 5 - Geração de Respostas

Gerando as respostas após o fine-tuning

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Elaborate a description of product based on the title provided.",
        "Golden Hatchet", # input
        "",
    )
], return_tensors = "pt").to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Elaborate a description of product based on the title provided.

### Title:
Golden Hatchet

### Content:
A golden hatchet that has been handcrafted from the finest materials. It has been designed to be both beautiful and functional, with a sharp blade and a comfortable handle. The hatchet is perfect for chopping wood, cutting through brush, and other outdoor tasks.

### Response:
A golden hatchet that has been handcrafted from the finest materials. It has been designed to be both beautiful and functional, with a sharp blade and a comfortable handle. The hatchet is perfect for chopping wood, cutting through brush, and other outdoor tasks. It is a great tool for anyone who loves the outdoors and wants to make the most of their time in


# Conclusão

Observamos que a resposta do modelo sem o fine-tuning é muito ampla, frequentemente apresentando informações irrelevantes ou desconexas, tornando difícil sua aplicação em cenários do mundo real. Em contraste, após o processo de fine-tuning, o modelo demonstrou uma compreensão mais refinada do contexto, resultando em respostas mais concisas, alinhadas com as necessidades específicas do domínio de interesse. Vale um destaque por algo que foi notado durante um dos promps para a criação de uma descrição do contepudo Golden Hatchet, em uma das execuções do modelo com o fining tuning realizado, criou a história de um livro. Isto ocorreu porque o dataset possui diversos livros e sinopses e, uma das ações de melhorias no dataset é ter uma classificação dos produtos para incluir no fining tuning, exemplo, livros, ferramentas, eletrodomésticos, etc.